In [7]:
import time
import pandas as pd

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# chromedriver PATH
chromedriver_path = '../chromedriver'
service = Service(chromedriver_path)
chrome_options = Options()
chrome_options.add_argument("--disable-images")
driver = webdriver.Chrome(service=service, options=chrome_options)

In [9]:
driver.get("https://www.google.com/maps/place/Cervecer%C3%ADa+17punto10/@41.640786,-0.8981089,15z/data=!4m6!3m5!1s0xd5914d93b5647a5:0xf566713a4aa5e5b!8m2!3d41.6407845!4d-0.8922415!16s%2Fg%2F11cn3d_jtr?entry=ttu&g_ep=EgoyMDI0MDkxOC4xIKXMDSoASAFQAw%3D%3D")
wait = WebDriverWait(driver, 5)

try:
    accept_button = driver.find_element(By.XPATH, "//button[@aria-label='Aceptar todo']")
    accept_button.click()
    print("Se hizo clic en 'Aceptar todo'")
    time.sleep(1)  # Wait
except Exception as e:
    # If button doesnt appear, continue
    pass

In [11]:
# Extract starts resume
wait = WebDriverWait(driver, 10)
reviews_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')))

raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')

print(raw_html_reviews[0].text)

raw_html_stars_resumme = raw_html_reviews[0].find_elements(By.CSS_SELECTOR, 'tr.BHOKXe')

ratings = []
reviews_counts = []

for star in raw_html_stars_resumme:
    rating_text = star.get_attribute('aria-label')
    rating_parts = rating_text.split(',')
    stars = rating_parts[0].split()[0] 
    num_reviews = rating_parts[1].strip().split()[0].replace('.', '')
    ratings.append(int(stars))
    reviews_counts.append(int(num_reviews))

stars_resumme = pd.DataFrame({'stars': ratings, 'reviews': reviews_counts})
display(stars_resumme)

5
4
3
2
1
4,6
823 reseñas
  Las reseñas no se verifican

  
Escribir una reseña
  

Ordenar
Todas
panceta
85
postres
56
croquetas
36
carpaccio
23
raviolis
12
burrata
11
tarta de manzana
9
canelón
7
langostino
7
decoración
7


,stars,reviews
0,5,571
1,4,191
2,3,35
3,2,13
4,1,13


In [14]:
# Extract all visible reviews

reviewers = []
ratings = []
review_texts = []
review_dates = []
local_guides = []
text_backups = []

# Init a set for process reviews
processed_review_ids = set()

SCROLL_PAUSE_TIME = 120

last_height = driver.execute_script("return arguments[0].scrollHeight", reviews_container)
while True:
    # Extract all visible reviews
    raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')
    
    # Extract data for each review
    for review in raw_html_reviews:
        try:
            review_id = review.get_attribute('data-review-id')
            
            # Verify if review has been processed
            if review_id in processed_review_ids:
                continue  # Omit processed review
            
            processed_review_ids.add(review_id)
            
            try:
                more_buttons = review.find_elements(By.CSS_SELECTOR, 'button.w8nwRe')
                for button in more_buttons:
                    if button.is_displayed():
                        button.click()
                        time.sleep(1)
            except Exception as e:
                print(f"No se encontró un botón 'Ver más' en esta reseña: {str(e)}")

            try: reviewer = review.find_element(By.CSS_SELECTOR, 'div.d4r55').text
            except: reviewer = ''

            try: review_text = review.find_element(By.CSS_SELECTOR, 'span.wiI7pd').text
            except: review_text = ''

            try: rating = review.find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label')
            except: rating = ''

            try: review_date = review.find_element(By.CSS_SELECTOR, 'span.rsqaWe').text
            except: review_date = ''

            try: local_guide = review.find_element(By.CLASS_NAME, "RfnDt").text
            except: local_guide = ''

            print(reviewer + ' - ' + rating + ' - ' + review_date + ' - ' + review_text) 

            reviewers.append(reviewer)
            review_texts.append(review_text)
            ratings.append(rating)
            review_dates.append(review_date)
            local_guides.append(local_guide)

            text_backups.append(review.text)

        except Exception as e:
            print(f"Error al extraer una reseña: {str(e)}")

    # Scroll down the page
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", reviews_container)
    time.sleep(10)

    new_height = driver.execute_script("return arguments[0].scrollHeight", reviews_container)
    print(new_height)
    print(last_height)
    # If height doesn't change, wait to load more reviews.
    if new_height == last_height:
        print('No more reviews detected, forcing additional scroll')
        # Wait more reviews to be loaded
        time.sleep(SCROLL_PAUSE_TIME) 
        new_raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')
        if len(new_raw_html_reviews) == len(raw_html_reviews): 
            print('No more reviews loaded, exiting scroll')
            break
    else:
        last_height = new_height

B Lana - 5 estrellas - Hace 2 meses - Espectacular!!! Probablemente uno de los restaurantes con mejor cocina de Zaragoza y a un precio mucho más que razonable. Probamos muchos platos (tomate en 4 texturas, carpaccio de presa, canelón de carne rustida, ravioli de setas, brandada de bacalao...) y todos estaban impresionantes. Nos encantaron también la sobrasada con miel casera y fritada de verduras de Aragón y la tarta de manzana del revés, super original. No dejéis de probarlas! Despista el nombre de cervecería, porque realmente aquí hay cocina. Ambiente muy acogedor y con buena música de fondo. Volveremos seguro! Gracias por hacernos disfrutar tanto :-)
Beatriz N. - 5 estrellas - Hace 2 meses - La comida es excepcional! Las migas, las croquetas, los langostinos en tempura, la panceta y los postres. Merece la pena probarlo!! El restaurante es muy sencillo en cuanto a decoración. Pero lo suplen con la comida tan rica que sirven. Repetiremos y recomendamos!!
Silvia Carbonell - 5 estrellas

In [ ]:
collected_data = pd.DataFrame({
    'author': reviewers,
    'local_guide_info': local_guides,
    'rating': ratings,
    'review': review_texts,
    'date': review_dates,
    'text_backup': text_backups
})
display(collected_data)

,author,local_guide_info,rating,review,date,text_backup
0,B Lana,35 reseñas · 245 fotos,5 estrellas,Espectacular!!! Probablemente uno de los resta...,Hace 2 meses,B Lana\n35 reseñas · 245 fotos\n\n\n\n\n\...
1,Beatriz N.,Local Guide · 28 reseñas · 8 fotos,5 estrellas,"La comida es excepcional! Las migas, las croqu...",Hace 2 meses,Beatriz N.\nLocal Guide · 28 reseñas · 8 fotos...
2,Silvia Carbonell,3 reseñas · 6 fotos,5 estrellas,Cenamos en familia y fue una experiencia gastr...,Hace un mes,Silvia Carbonell\n3 reseñas · 6 fotos\n\n\n...
3,Carlos Casado,16 reseñas · 30 fotos,5 estrellas,Espacio confortable aún estando en la barra ja...,Hace 4 meses,Carlos Casado\n16 reseñas · 30 fotos\n\n\n\...
4,Carolina Gonzalez,Local Guide · 54 reseñas · 66 fotos,5 estrellas,SIEMPRE DE 10!\nHemos venido varias veces y ta...,Hace 3 semanas,Carolina Gonzalez\nLocal Guide · 54 reseñas · ...
5,NMJ,Local Guide · 158 reseñas · 406 fotos,5 estrellas,Fuimos recomendados por un amigo y fue todo un...,Hace 6 meses,NMJ\nLocal Guide · 158 reseñas · 406 fotos\n\...
6,BEATRIZ SANJOAQUIN,Local Guide · 103 reseñas · 23 fotos,5 estrellas,Un sitio muy interesante para visitar. El serv...,Hace 5 meses,BEATRIZ SANJOAQUIN\nLocal Guide · 103 reseñas ...
7,Cristina Q,9 reseñas · 14 fotos,5 estrellas,Sitio de nivel. La comida relación calidad-pre...,Hace 8 meses,Cristina Q\n9 reseñas · 14 fotos\n\n\n\n\n...
8,Beatriz CD,Local Guide · 40 reseñas · 48 fotos,5 estrellas,"Fuimos de celebración,el sitio es pequeño, de ...",Hace 6 meses,Beatriz CD\nLocal Guide · 40 reseñas · 48 foto...
9,Daniel Sánchez-Rubio,Local Guide · 64 reseñas · 111 fotos,4 estrellas,Es un sitio donde la comida es sencillamente d...,Hace un año,Daniel Sánchez-Rubio\nLocal Guide · 64 reseñas...


In [ ]:
name = '17punto10.csv'

csv_file_path = '../data/raw/collected_reviews_'
collected_data.to_csv(csv_file_path + name, index=False)

stars_resumme_path = '../data/raw/resumme_'
collected_data.to_csv(stars_resumme_path + name, index=False)